# ReAct Agent Tutorial

This notebook demonstrates the **ReAct (Reasoning + Acting)** pattern:

1. **Think**: The agent reasons about what it needs to do
2. **Act**: The agent calls a tool
3. **Observe**: The agent receives the tool's result
4. **Repeat**: Until the agent has enough information

```
User Question → Think → Act → Observe → Think → ... → Final Answer
```

## Setup

In [ ]:
# Install miiflow-llm
!pip install -q miiflow-llm nest_asyncio

import nest_asyncio
nest_asyncio.apply()

In [ ]:
import os
from getpass import getpass

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

print("API key configured!")

## Define Tools

Tools are functions the agent can call. The `@tool` decorator registers a function as a tool.

In [ ]:
import math
from miiflow_llm.core.tools import tool


@tool("calculate", "Evaluate mathematical expressions")
def calculate(expression: str) -> str:
    """Safely evaluate a math expression.
    
    Args:
        expression: A mathematical expression like '2 + 2' or 'sqrt(16)'
    """
    allowed = {
        "abs": abs, "round": round, "min": min, "max": max,
        "sqrt": math.sqrt, "pow": pow, "pi": math.pi, "e": math.e
    }
    try:
        result = eval(expression, {"__builtins__": {}}, allowed)
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"


@tool("get_weather", "Get current weather for a location")
def get_weather(location: str) -> str:
    """Get weather information for a location.
    
    Args:
        location: City name
    """
    weather_data = {
        "new york": "Sunny, 72°F (22°C)",
        "london": "Cloudy, 59°F (15°C)",
        "tokyo": "Rainy, 68°F (20°C)",
        "paris": "Partly cloudy, 65°F (18°C)",
    }
    location_lower = location.lower()
    if location_lower in weather_data:
        return f"Weather in {location}: {weather_data[location_lower]}"
    return f"Weather data not available for {location}"


print("Tools defined: calculate, get_weather")

## Example 1: Basic Usage

Create a ReAct agent and ask a question that requires using tools.

In [ ]:
from miiflow_llm import LLMClient, Agent, AgentType

# Create client and agent
client = LLMClient.create("openai", model="gpt-4o-mini")

agent = Agent(
    client,
    agent_type=AgentType.REACT,
    max_iterations=5,
    system_prompt="You are a helpful assistant. Use tools to answer questions.",
)

agent.add_tool(calculate)
agent.add_tool(get_weather)

print("Agent created with 2 tools!")

In [ ]:
# Ask a question that requires tools
result = await agent.run("What is sqrt(144) + 8?")
print(result.data)

In [ ]:
# Ask about weather
result = await agent.run("What's the weather like in Tokyo and Paris?")
print(result.data)

## Example 2: Streaming

Watch the agent's reasoning in real-time by streaming events.

In [ ]:
from miiflow_llm import RunContext
from miiflow_llm.core.react import ReActEventType

print("Query: What's the weather in London and calculate 25 * 4?")
print("=" * 50)

context = RunContext(deps=None, messages=[])

async for event in agent.stream("What's the weather in London and calculate 25 * 4?", context):
    if event.event_type == ReActEventType.THINKING_CHUNK:
        print(event.data.get("delta", ""), end="", flush=True)
    
    elif event.event_type == ReActEventType.ACTION_PLANNED:
        action = event.data.get("action", "")
        print(f"\n\n[Calling tool: {action}]\n")
    
    elif event.event_type == ReActEventType.OBSERVATION:
        obs = event.data.get("observation", "")[:100]
        print(f"[Tool result: {obs}]\n")
    
    elif event.event_type == ReActEventType.FINAL_ANSWER:
        print("\n" + "=" * 50)
        print("FINAL ANSWER:")
        print(event.data.get("answer", ""))

## Key Takeaways

1. **ReAct Pattern**: Think → Act → Observe → Repeat until done

2. **When to use ReAct**:
   - Simple to moderately complex queries
   - Questions requiring tool lookups
   - Tasks where steps are unknown upfront

3. **Key parameters**:
   - `agent_type=AgentType.REACT` - enables ReAct mode
   - `max_iterations` - limits reasoning cycles

## Next Steps

For complex tasks with multiple subtasks, check out the **Plan & Execute** tutorial.